In [1]:
import numpy as np
import math
MAX_DIGITS = 16 

In [2]:
def sigmoid(x):  
    s = 1 / (1 + np.exp(-x))  
    return s  

def decompose(x):
    y =[]
    for i in reversed(range(MAX_DIGITS)):
        y.append(int(x/math.pow(2,i)))
        if (x >= math.pow(2,i)):
            x -= math.pow(2,i)
    return y

def compose(Y):
    z = 0
    for i in range(MAX_DIGITS):
        z += int(math.pow(2, MAX_DIGITS - 1 - i) * Y[i])
    return z

In [ ]:
class RNN_Addition(object):
        
    def __init__(self):
        pass

    def train(self, num_hidden = 32, learning_rate=0.5, num_iters=100000):
        
        self.num_hidden = num_hidden
        self.parameters = {}
        self.grads = {}
        
        self.parameters['Wax'] = 2 * np.random.random((2, num_hidden)) - 1  
        self.parameters['Wya'] = 2 * np.random.random((num_hidden, 1)) - 1  
        self.parameters['Waa'] = 2 * np.random.random((num_hidden, num_hidden)) - 1  
  
        self.grads['Wax'] = np.zeros_like(self.parameters['Wax'])  
        self.grads['Wya'] = np.zeros_like(self.parameters['Wya'])  
        self.grads['Waa'] = np.zeros_like(self.parameters['Waa']) 
        
        for i in range(num_iters):
            
            input_1 = np.random.randint(0,10000) 
            input_2 = np.random.randint(0,10000) 
            x_1 = decompose(input_1)  
            x_2 = decompose(input_2)  
            y = decompose(input_1 + input_2)   

            loss = 0  

            dY_hat = []  
            A = [] 
            A.append(np.zeros(num_hidden))  

            for j in range(MAX_DIGITS):

                X = np.array([[x_1[MAX_DIGITS - j - 1], x_2[MAX_DIGITS - j - 1]]])  
                Y = np.array([[y[MAX_DIGITS - j - 1]]]).T  
                A_temp = sigmoid(np.dot(X, self.parameters['Wax']) + np.dot(A[-1], self.parameters['Waa']))  
                Y_hat = sigmoid(np.dot(A_temp, self.parameters['Wya']))  

                dY_hat.append((Y - Y_hat) * Y_hat * (1-Y_hat)) 
                loss += np.abs(Y - Y_hat[0])   

                A.append(A_temp)  
        
            dA_plus = np.zeros(num_hidden)  
  
            for j in range(MAX_DIGITS):  
                X = np.array([[x_1[j], x_2[j]]])  
                A_temp = A[-j - 1]  

                A_prev_temp = A[-j - 2]  

                dY_hat_temp = dY_hat[-j - 1]  
                dA_temp = (dA_plus.dot(self.parameters['Waa'].T) + dY_hat_temp.dot(  
                    self.parameters['Wya'].T)) * A_temp * (1-A_temp)  

                self.grads['Wya'] += np.atleast_2d(A_temp).T.dot(dY_hat_temp)
                self.grads['Waa'] += np.atleast_2d(A_prev_temp).T.dot(dA_temp) 
                self.grads['Wax'] += X.T.dot(dA_temp) 

                dA_plus = dA_temp  

            self.parameters['Wax'] += self.grads['Wax'] * learning_rate  
            self.parameters['Waa'] += self.grads['Waa'] * learning_rate  
            self.parameters['Wya'] += self.grads['Wya'] * learning_rate  

            self.grads['Wax'] *= 0  
            self.grads['Waa'] *= 0  
            self.grads['Wya'] *= 0  

            if (i % 1000 == 0): 
                print('After '+str(i)+' iterations, error: '+str(loss))
                
        return
    
    #####################################################################################
    
    def calculate(self, input_1, input_2):
        
        A = list()  
        A.append(np.zeros(self.num_hidden))  
        X_1 = decompose(input_1)  
        X_2 = decompose(input_2) 
        Y_hat_bin = np.zeros(MAX_DIGITS) 
  
        for position in range(MAX_DIGITS):  
                # generate input and output  
            X = np.array([[X_1[MAX_DIGITS - position - 1], X_2[MAX_DIGITS - position - 1]]])  
  
            A_temp = sigmoid(np.dot(X, self.parameters['Wax']) + np.dot(A[-1], self.parameters['Waa']))  
  
            Y_hat_temp = sigmoid(np.dot(A_temp, self.parameters['Wya']))  
  
            Y_hat_bin[MAX_DIGITS - position - 1] = np.round(Y_hat_temp[0][0])  
            A.append(A_temp)

        output = compose(Y_hat_bin)
        return output
    
    def evaluate(self):
        correct = 0
        for i in range(1000000):
            input_1 = np.random.randint(0,10000) 
            input_2 = np.random.randint(0,10000) 
            output = self.calculate(input_1, input_2)
            if output == input_1 + input_2:
                correct += 1
            if i % 10000 == 0:
                print(str(correct)+' additions correct out of '+str(i))
        accuracy = correct / (1000000)
        print(accuracy)
        return accuracy

In [ ]:
addtion = RNN_Addition()
addtion.train()
addtion.evaluate()

After 0 iterations, error: [[ 7.91319623]]
After 1000 iterations, error: [[ 2.93205376]]
After 2000 iterations, error: [[ 0.53235364]]
After 3000 iterations, error: [[ 0.42437695]]
After 4000 iterations, error: [[ 0.1850921]]
After 5000 iterations, error: [[ 0.14446907]]
After 6000 iterations, error: [[ 0.18902951]]
After 7000 iterations, error: [[ 0.15777253]]
After 8000 iterations, error: [[ 0.12779185]]
After 9000 iterations, error: [[ 0.1468772]]
After 10000 iterations, error: [[ 0.13331396]]
After 11000 iterations, error: [[ 0.11130219]]
After 12000 iterations, error: [[ 0.15254127]]
After 13000 iterations, error: [[ 0.09677258]]
After 14000 iterations, error: [[ 0.13692135]]
After 15000 iterations, error: [[ 0.09687071]]
After 16000 iterations, error: [[ 0.09238099]]
After 17000 iterations, error: [[ 0.07698048]]
After 18000 iterations, error: [[ 0.10517827]]
After 19000 iterations, error: [[ 0.08506512]]
After 20000 iterations, error: [[ 0.07119982]]
After 21000 iterations, erro